In [1]:
#Bollywood Data Downloader 
# 25th July 2025

#Importing and Calling Packages 
import os
import time
import requests
import pandas as pd
from tqdm import tqdm

#Keys, File Paths and Passwords 
#NOTE: Contains API Keys and Passwords 
tmdb_api_key = "ef74866feee9084817794614ffbae21d"
tmdb_user_agent = "RAApp/1.0.0"

opensub_api_key = "V8cxCT3494611SyHgQRVHbetkQEZVTHW"
opensub_username = "rishabh_bijani"
opensub_password = "Rishabhb2004!"  

#PLEASE CHANGE FILE PATH ACCORDINGLY 
# PLEASE ENSURE - movies_random_sample.csv is in Desktop/ra_app
csv_path = os.path.expanduser("~/Desktop/ra_app/movies_random_sample.csv")



base_folder = os.path.expanduser("~/Desktop/ra_app/data")
log_file = os.path.join(base_folder, "log_downloads.txt")

#Creating Folders 
os.makedirs(os.path.join(base_folder, "description"), exist_ok=True)
os.makedirs(os.path.join(base_folder, "posters"), exist_ok=True)
os.makedirs(os.path.join(base_folder, "subtitles"), exist_ok=True)

#Logging Function
def log(msg):
    print(msg)
    with open(log_file, "a", encoding="utf-8") as f:
        f.write(msg + "\n")

#Login into OpenSubtitles 
log("🔐 Logging in to OpenSubtitles...")
login_url = "https://api.opensubtitles.com/api/v1/login"
headers_login = {
    "Api-Key": opensub_api_key,
    "Content-Type": "application/json",
    "User-Agent": tmdb_user_agent
}
payload = {
    "username": opensub_username,
    "password": opensub_password
}

login_resp = requests.post(login_url, headers=headers_login, json=payload)
if login_resp.status_code == 200:
    opensub_token = login_resp.json()["token"]
    log("✅ OpenSubtitles login successful.")
else:
    log("❌ OpenSubtitles login failed: " + login_resp.text)
    raise Exception("Cannot proceed without OpenSubtitles token.")

#Download Loop
df = pd.read_csv(csv_path)
imdb_ids = df["imdb_id"].dropna().unique()
report_rows = []

headers_auth = {
    "Authorization": f"Bearer {opensub_token}",
    "Api-Key": opensub_api_key,
    "Content-Type": "application/json",
    "Accept": "application/json",
    "User-Agent": tmdb_user_agent
}

for imdb_id in tqdm(imdb_ids):
    log(f"\n=== Processing {imdb_id} ===")
    desc_status = poster_status = sub_status = "❌"

    # --- TMDb DESCRIPTION ---
    try:
        tmdb_url = f"https://api.themoviedb.org/3/find/{imdb_id}?api_key={tmdb_api_key}&external_source=imdb_id"
        tmdb_resp = requests.get(tmdb_url)
        if tmdb_resp.status_code == 200:
            movie = tmdb_resp.json().get("movie_results", [{}])[0]
            overview = movie.get("overview", "")
            desc_path = os.path.join(base_folder, "description", f"{imdb_id}.txt")
            with open(desc_path, "w", encoding="utf-8") as f:
                f.write(overview)
            desc_status = "✅"
            log("✅ Description saved.")
        else:
            log(f"❌ TMDb description failed (status {tmdb_resp.status_code})")
    except Exception as e:
        log(f"❌ TMDb description error: {e}")

    # --- TMDb POSTER ---
    try:
        poster_path = movie.get("poster_path", "")
        if poster_path:
            poster_url = f"https://image.tmdb.org/t/p/original{poster_path}"
            poster_resp = requests.get(poster_url)
            if poster_resp.status_code == 200:
                poster_file = os.path.join(base_folder, "posters", f"{imdb_id}.png")
                with open(poster_file, "wb") as f:
                    f.write(poster_resp.content)
                poster_status = "✅"
                log("✅ Poster saved.")
            else:
                log("❌ Poster download failed.")
        else:
            poster_status = "⚠️"
            log("⚠️ No poster path available.")
    except Exception as e:
        log(f"❌ Poster error: {e}")

    # --- OpenSubtitles SUBTITLES ---
    try:
        search_url = f"https://api.opensubtitles.com/api/v1/subtitles?imdb_id={imdb_id[2:]}&languages=en"
        sub_resp = requests.get(search_url, headers=headers_auth)
        if sub_resp.status_code == 200:
            results = sub_resp.json().get("data", [])
            if results:
                file_id = results[0]["attributes"]["files"][0]["file_id"]
                dl_resp = requests.post(
                    "https://api.opensubtitles.com/api/v1/download",
                    headers=headers_auth,
                    json={"file_id": file_id}
                )
                if dl_resp.status_code == 200:
                    dl_link = dl_resp.json()["link"]
                    srt_resp = requests.get(dl_link)
                    if srt_resp.status_code == 200:
                        sub_path = os.path.join(base_folder, "subtitles", f"{imdb_id}.srt")
                        with open(sub_path, "wb") as f:
                            f.write(srt_resp.content)
                        sub_status = "✅"
                        log("✅ Subtitle saved.")
                    else:
                        log("❌ Subtitle download failed.")
                else:
                    log("❌ Subtitle download link error.")
            else:
                sub_status = "⚠️"
                log("⚠️ No subtitles found.")
        else:
            log("❌ Subtitle search failed.")
    except Exception as e:
        log(f"❌ Subtitle error: {e}")

    # --- Append to Report ---
    report_rows.append({
        "imdb_id": imdb_id,
        "description_status": desc_status,
        "poster_status": poster_status,
        "subtitle_status": sub_status
    })

#CSV Download Report 
report_df = pd.DataFrame(report_rows)
report_df.to_csv(os.path.join(base_folder, "download_report.csv"), index=False)
log("📄 Saved report to download_report.csv")
log("✅ Script finished.")

🔐 Logging in to OpenSubtitles...
✅ OpenSubtitles login successful.


  0%|                                                   | 0/100 [00:00<?, ?it/s]


=== Processing tt8207768 ===
✅ Description saved.
✅ Poster saved.


  1%|▍                                          | 1/100 [00:04<07:00,  4.25s/it]

✅ Subtitle saved.

=== Processing tt6903440 ===
✅ Description saved.
✅ Poster saved.


...

✅ Subtitle saved.

=== Processing tt4309284 ===
✅ Description saved.
⚠️ No poster path available.


 99%|█████████████████████████████████████████▌| 99/100 [07:44<00:04,  4.48s/it]

✅ Subtitle saved.

=== Processing tt6334874 ===
✅ Description saved.
✅ Poster saved.


100%|█████████████████████████████████████████| 100/100 [07:49<00:00,  4.69s/it]

✅ Subtitle saved.
📄 Saved report to download_report.csv
✅ Script finished.
